# Introduction

The idea behind nornir_tests is to allow for adding tests against the task run and its results.  Many times, a task can run and get some output but that doesn't mean it is successful.  One option is to add additional checks or tasks with some conditional logic to deal with this.  Another is to add tests against the data returned from a task.  That is what nornir_tests is for.

The question is probably why use tests.  Tests were inspired by the same feature in postman.  In postman, a test was something that was done against the data returned from a postman request to determine if the request was actually successful.  This can certainly be added in python using either pure python or additional tasks but there is some elegance and some additional functionality obtained by using tests tied to tasks.  In this intro, I will try to touch on a variety of ideas using tests.  In further tutorials I'll go into more detail on using each of the test types.

Typical nornir initialization is done first.

In [1]:
from nornir_napalm.plugins.tasks import napalm_get, napalm_ping
from nornir_utils.plugins.functions import print_result
from nornir import InitNornir

nr = InitNornir(
    inventory={
        "plugin": "SimpleInventory",
        "options": {
            "host_file": "data/hosts.yaml",
            "group_file": "data/groups.yaml",
            "defaults_file": "data/defaults.yaml",
        },
    },
)

Next the TestsProcessor is appended to the processors list and the inventory is shown. This can be done using config.yaml as well or a few other ways that are covered in the Nornir documentation.

In [2]:
from nornir_tests.plugins.processors import TestsProcessor

nr.processors.append(TestsProcessor())

print(nr.inventory.hosts)

{'test': Host: test, 'vyos': Host: vyos}


In the first example, a vyos router interface facts are obtained using napalm and a test is run to validate the result.

In [3]:
from nornir_tests.plugins.tests import test_regexp, test_until, test_timing, test_jsonpath

vyos = nr.filter(name='vyos')

result = vyos.run(
    task=napalm_get,
    getters=['interfaces'],
    tests=[
        test_jsonpath(path='interfaces.eth0.is_up', value=True),
    ]
)

print_result(result, vars=['result', 'tests'])

napalm_get**********************************************************************
* vyos ** changed : False ******************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{'interfaces':{'eth0':{'description':'',
'is_enabled':True,
'is_up':True,
'last_flapped':-1.0,
'mac_address':'08:00:27:e0:28:63',
'mtu':-1,
'speed':0},
'lo':{'description':'',
'is_enabled':True,
'is_up':True,
'last_flapped':-1.0,
'mac_address':'00:00:00:00:00:00',
'mtu':-1,
'speed':0}}}
('jsonpath: interfaces.eth0.is_up expanded to interfaces.eth0.is_up and value '
 'True found - PASSED')

^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


Next a ping is sent to the vyos device using the timing test.  As can be seen in the output, this gives runtime of the task and can be enhanced with min_run_time and max_run_time to affect the task result if the fail_task argument is used.  If a failure occurs in the test and fail_task is set to True, then the task will be marked as failed.

In [4]:
result = vyos.run(
    task=napalm_ping,
    dest='192.168.99.1',
    tests=[
        test_timing(),
    ]
)
print_result(result, vars=['tests', 'result'])

napalm_ping*********************************************************************
* vyos ** changed : False ******************************************************
vvvv napalm_ping ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
"('timing: 0 < 4.914697647094727 < 9223372036854775807',) - PASSED"

{'success':{'packet_loss':0,
'probes_sent':5,
'results':[{'ip_address': '192.168.99.1', 'rtt': 4.84}],
'rtt_avg':4.84,
'rtt_max':8.166,
'rtt_min':2.693,
'rtt_stddev':1.962}}
^^^^ END napalm_ping ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


The next example shows how the result failed state can be affected by a test.

In [5]:
result = vyos.run(
    task=napalm_ping,
    dest='192.168.99.1',
    tests=[
        test_timing(max_run_time=1, fail_task=True),
    ]
)
print_result(result, vars=['tests', 'result'])

napalm_ping*********************************************************************
* vyos ** changed : False ******************************************************
vvvv napalm_ping ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv ERROR
"('timing: 0 < 4.8129801750183105 < 1',) - FAILED"

{'success':{'packet_loss':0,
'probes_sent':5,
'results':[{'ip_address': '192.168.99.1', 'rtt': 3.349}],
'rtt_avg':3.349,
'rtt_max':4.976,
'rtt_min':2.689,
'rtt_stddev':0.829}}
^^^^ END napalm_ping ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


Tests can be stacked and the entire task and associated tests can be retried using test_until.  The next test shows how a rebooted device can be retried until it comes back up using only tests.  As napalm for vyos doesn't currently support CLI, I rebooted it manually and ran the ping below to show it was down.

In [8]:
nr.data.reset_failed_hosts()

result = vyos.run(
    task=napalm_ping,
    dest='192.168.99.1',
)
print_result(result, vars=['tests', 'result'])

napalm_ping*********************************************************************
* vyos ** changed : False ******************************************************
vvvv napalm_ping ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv ERROR
Traceback (most recent call last):
  File "/home/patrick/dev/nornir_tests/.venv/lib/python3.8/site-packages/nornir/core/task.py", line 98, in start
    r = self.task(self, **self.params)
  File "/home/patrick/dev/nornir_tests/.venv/lib/python3.8/site-packages/nornir_napalm/plugins/tasks/napalm_ping.py", line 37, in napalm_ping
    result = device.ping(
  File "/home/patrick/dev/nornir_tests/.venv/lib/python3.8/site-packages/napalm_vyos/vyos.py", line 857, in ping
    output_ping = self.device.send_command(command)
  File "/home/patrick/dev/nornir_tests/.venv/lib/python3.8/site-packages/netmiko/utilities.py", line 363, in wrapper_decorator
    return func(self, *args, **kwargs)
  File "/home/patrick/dev/nornir_tests/.venv/lib/python3.8/site-p

After rebooting and failing the ping above, the get command is started with test_until and test_jsonpath.  test_jsonpath was set to fail the task so that test_until would continue to loop.

In [9]:
nr.data.reset_failed_hosts()
result = vyos.run(
    task=napalm_get,
    getters=['interfaces'],
    tests=[
        test_jsonpath(path='interfaces.eth0.is_up', value=True, fail_task=True),
        test_until(initial_delay=15, retries=10, delay=15, reset_conns=True),
    ]
)
print_result(result, vars=['tests', 'result'])

napalm_get**********************************************************************
* vyos ** changed : False ******************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
('jsonpath: interfaces.eth0.is_up expanded to interfaces.eth0.is_up and value '
 'True found - PASSED')
'until: succeeded after 36.63877630233765 seconds - PASSED'

{'interfaces':{'eth0':{'description':'',
'is_enabled':True,
'is_up':True,
'last_flapped':-1.0,
'mac_address':'08:00:27:e0:28:63',
'mtu':-1,
'speed':0},
'lo':{'description':'',
'is_enabled':True,
'is_up':True,
'last_flapped':-1.0,
'mac_address':'00:00:00:00:00:00',
'mtu':-1,
'speed':0}}}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


Not bad.  The result above shows that it must have failed on its first try but by 36 seconds it was able to pass the test_jsonpath test and the call to napalm_get and so test_until finished up.

This was pretty haphazard but other tutorials will go into more details about each of the tests.